# Regression. Part 2

---
Author: Durkin Anatoliy

Updated: 31.03.2025

---
В данном ноутбуке ...

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
def metrics(true, pred):
    print('R2:', r2_score(true, pred))
    print('MAE:', mean_absolute_error(true, pred))
    print('RMSE:', mean_squared_error(true, pred)**0.5)

In [ ]:
df = pd.read_csv('house_price_regression_dataset.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['House_Price'], axis=1), df['House_Price'], test_size=0.2, random_state=42)

# Модели

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression().fit(X_train, y_train)

In [ ]:
metrics(y_test, lr.predict(X_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor().fit(X_train, y_train)

In [ ]:
metrics(y_test, rf.predict(X_test))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbr = GradientBoostingRegressor().fit(X_train, y_train)

In [ ]:
metrics(y_test, gbr.predict(X_test))

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cb = CatBoostRegressor().fit(X_train, y_train, verbose=False)

In [ ]:
metrics(y_test, cb.predict(X_test))

# Отбор признаков

## Прямой отбор

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

In [ ]:
cb = CatBoostRegressor(verbose=False)
sfs = SequentialFeatureSelector(cb, direction='forward')
sfs.fit(X_train, y_train)

In [ ]:
sfs.get_support()

In [ ]:
sfs.get_params()

In [ ]:
sfs.transform(X_test)

## Последовательный отбор

In [ ]:
cb = CatBoostRegressor(verbose=False)
sfs = SequentialFeatureSelector(cb, direction='backward')
sfs.fit(X_train, y_train)

In [ ]:
sfs.get_support()

## Исчерпывающий выбор

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector

In [ ]:
cb = CatBoostRegressor(verbose=False)
efs = ExhaustiveFeatureSelector(cb, min_features=1, max_features=7, scoring='r2', cv=5)
efs.fit(X_train, y_train)

In [ ]:
efs.best_score_

In [ ]:
efs.best_feature_names_

In [ ]:
efs.subsets_

# Pipeline

## Pipeline as transformer

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
simple_imputer = SimpleImputer(strategy='mean')
scaler = MinMaxScaler()

In [ ]:
pipe = Pipeline(steps=[('imputer', simple_imputer), ('scaler', scaler)])

In [ ]:
pipe.fit(X_train)

In [ ]:
pipe.transform(X_test)

## Pipeline as model

In [ ]:
model = LinearRegression()

In [ ]:
pipe = Pipeline(steps=[('imputer', simple_imputer), ('scaler', scaler), ('model', model)])

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
metrics(y_test, pipe.predict(X_test))

## Обработка разнородных данных

In [ ]:
df = pd.read_csv('insurance.csv')

In [ ]:
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['charges'], axis=1), df['charges'], test_size=0.2, random_state=42)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
col_transformer = ColumnTransformer([('num_preproc', MinMaxScaler(), [x for x in X_train.columns if X_train[x].dtype!='object']),
                                     ('cat_preproc', OneHotEncoder(dtype='int'), [x for x in X_train.columns if X_train[x].dtype=='object'])])

In [ ]:
pipe = Pipeline([('preproc', col_transformer), ('LR', LinearRegression())])

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
metrics(y_test, pipe.predict(X_test))

## Подбор гиперпараметров

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
pipe = Pipeline([('preproc', col_transformer), ('CatBoost', CatBoostRegressor(verbose=False))])

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)

In [ ]:
param_grid = {
    "CatBoost__iterations": [1000, 2000],
    "CatBoost__learning_rate": [0.01, 0.05],
    "CatBoost__depth": [3, 5, 7]
}
search = GridSearchCV(pipe, param_grid)

In [ ]:
search.fit(X_train, y_train)

In [ ]:
search.best_score_

In [ ]:
search.best_params_